In [ ]:
# ENV["PROJECT_PATH_ED"]="../envs/KED"

# include("../src/mybase.jl")
using KaiEDJ
using KaiEDJ: BenchmarkTools, Optimization, Optim, Plots
using DelimitedFiles



######## Correlated Orbital Info #########
norb    = 1
nspin   = 2
nspinorb    = norb * nspin

# IndOrbUp, IndOrbDn = GetOrbUpDn( nspinorb )
IndOrbUp    = [ i for i in 1:2:nspinorb ]
IndOrbDn    = [ i for i in 2:2:nspinorb ]


######## Imaginary Frequency Green Function Construction #########
beta    = 100
NImFreq  = 4*beta
ImFreqGridVal   = GetImFreqValGrid( beta, NImFreq )
ImFreqGrid      = ImFreqGridVal * im
# Hybiwup_init  = 1. / 4 * GetGzBetheUniformScaling.( ImFreqGrid )
# Hybiwdn_init  = 1. / 4 * GetGzBetheUniformScaling.( ImFreqGrid )
D   = 1
gbetheiw    = GetGzBetheDim.( ImFreqGrid, nspinorb ; D=D )
gbetheiwup  = GetGzBetheUniformScaling.( ImFreqGrid ; D=D )
Hybiw       = 1. / 4 * GetGzBetheDim.( ImFreqGrid, nspinorb ; D=D )

In [ ]:
######## Bath Orbital Info #########
nbath   = 6
ntot    = nspinorb + nbath
dim     = 2^ntot
@show dim
nbathHalf   = div(nbath,2)
# IndBathUp, IndBathDn = GetOrbUpDn( nbath )
IndBathUp   = [ i for i in 1:2:nbath ]
IndBathDn   = [ i for i in 2:2:nbath ]

ebathl     = zeros(nbath)
ebathl     = collect(LinRange( -1, 1, nbath ))
@show ebathl

Vil     = zeros(nspinorb,nbath)
for i in 1:nspinorb
    for j in 1:nbath
        Vil[i,j]    = ebathl[j]
    end
end
println( "Vil : " ) ; writedlm(stdout, Vil)

In [ ]:
######## Bath Discretization Setup (Spin-up/dn) #########
ebathlnew, Vilnew   = BathDiscHybSpinPH( ebathl, Vil, Hybiw, ImFreqGrid )

In [ ]:
ShowBathParam( ebathlnew, Vilnew )

In [ ]:
######## Setting Up Hamiltonian Operators #########
U           = 0
JHund       = 0
chem        = 0.5*U # 0.5*U for single-band, 2.5*U-5.0*JHund for t2g-multiband
opcavec     = [ GetOpBathParam(ebathlnew, Vilnew, ibath->ibath+nspinorb),
                GetOpChemPot(chem, nspinorb) ]
opccaavec   = [ GetOpUSlaterKanamori( ; U=U, JHund=JHund, norb=norb ) ]
# @show typeof(opcavec)
# @show typeof(opccaavec)
# @show opcavec
# @show opccaavec

######## Fock/Hilbert Space Construction #########
outputlevel = 0

######## Searching Ground-sector #########
Emin_arr    = SearchGSSector( ntot, opcavec, opccaavec ; outputlevel=1 )

In [ ]:
@show Emin_arr

In [ ]:
######## Choosing Ground-sectors #########
# IndGSSector = [ [ isector, e0, boltzweight ] ]
iGSSector   = argmin(Emin_arr)

In [ ]:
######## Accurate Hamiltonian Diagonalization (for the choosen sectors) #########
esyssec_AR = GetGSFromSector( iGSSector, ntot, opcavec, opccaavec ; outputlevel=1 )

In [ ]:
######## Boltzmann Weight #########
# IndGSSector = [ [ isector, e0, boltzweight, evec ] ]
ieval        = 1
wBoltz = 1.0 
IndGSSector = [ [ iGSSector, esyssec_AR[1][ieval], wBoltz, esyssec_AR[2][:,ieval] ] ]

In [ ]:
######## Impurity Green Function Construction #########
Gimp    = GetGreenImpurityFromGS(  nspinorb, IndGSSector[1], ntot, opcavec, opccaavec, ImFreqGrid )
gimpup  = GetijarrayFromVecMat( Gimp, 1, 1 )
gimpdn  = GetijarrayFromVecMat( Gimp, 2, 2 )

G0imp   = GetGreenDiscGrid( ebathlnew, Vilnew, ImFreqGrid, KaiEDJ.I(nspinorb)*chem )
g0impup = GetijarrayFromVecMat( G0imp, 1, 1 )
g0impdn = GetijarrayFromVecMat( G0imp, 2, 2 )

In [ ]:

Plots.plot( ImFreqGridVal, imag(gimpup) , lt=:scatter, marker=:square, label="gimpup")
Plots.plot!( ImFreqGridVal, imag(gimpdn) , lt=:scatter, marker=:circle, label="gimpdn")
Plots.plot!( ImFreqGridVal, imag(g0impup), linewidth=2, label="g0impup")
Plots.plot!( ImFreqGridVal, imag(g0impdn), linewidth=3, line=:dash, label="g0impdn")
Plots.xlabel!("Imaginary Frequency")
Plots.ylabel!("Green's Function")
Plots.title!("Green's Functions on Imaginary Frequency Grid")


#Plots.savefig("14_dmft_ed_solver_bethe.png")


In [ ]:
selfup  = GetSelf( g0impup, gimpup )
selfdn  = GetSelf( g0impdn, gimpdn )

G0newimp    = GetGreenImpGrid( zeros(nspinorb,nspinorb), (D*D / 2. / 2. ) * Gimp, ImFreqGrid )

In [ ]:
# using Plots
Plots.plot( ImFreqGridVal, imag(selfup) )
Plots.plot!( ImFreqGridVal, imag(gimpdn) , marker=:circle)
Plots.plot!( ImFreqGridVal, imag(g0impup) , marker=:square)